# Coleta de Tweets

Data: 06/03/2019 (updated) <br/>
Autor: Diego Abreu | https://github.com/DiegoAbreu/

***

## Conexão com o Twitter:

### Pacote:

In [ ]:
# Instalação do pacote Tweepy
!pip install tweepy

In [ ]:
# Importação de módulos
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

### Chaves da API do Twitter:

In [ ]:
# Consumer Key
consumer_key = "xxxxx"
# Consumer Secret 
consumer_secret = "xxxxx"
# Access Token
access_token = "xxxxx"
# Access Token Secret
access_token_secret = "xxxxx"

In [ ]:
# Chaves de autenticação
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

### Classe para captura dos tweets e armazenamento no banco de dados

In [ ]:
# Classe de captura de stream
class MyListener(StreamListener):
    def on_data(self, data):
        tweet = json.loads(data)
        
        created_at = tweet["created_at"]
        id_str = tweet["id_str"]
        screen_name = tweet["user"]["screen_name"]
        followers_count = tweet["user"]["followers_count"]
        text = tweet["text"]
        
        obj = {"created_at":created_at,"id_str":id_str,
               "screen_name":screen_name, "followers_count":followers_count,
               "text":text,}
        
        tweetind = col.insert_one(obj).inserted_id
        print (obj)
        return True

In [ ]:
# Objeto mylistener
mylistener = MyListener()
# Objeto mystream
mystream = Stream(auth, listener = mylistener)

***

## Conexão com o MongoDB

In [ ]:
# Importação do módulo MongoClient
from pymongo import MongoClient 

In [ ]:
# Conexão com MongoDB
client = MongoClient('localhost', 27017)

In [ ]:
# Criação do banco de dados
db = client.twitterdb
# Criando da coleção
col = db.tweets 

In [ ]:
# Lista de palavras chave para buscar nos Tweets
keywords = ['xxxxxx', 'yyyyyy', 'zzzzzz']

***

## Coleta de Tweets

In [ ]:
# Acionamento do filtro de tweets e gravação no MongoDB
mystream.filter(track=keywords)
# Pausar a execução para interromper a coleta

In [ ]:
mystream.disconnect()

***

## Consulta de Dados no MongoDB

In [ ]:
# consulta de um elemento da coleção
col.find_one()

In [ ]:
# criação de um dataset com dados do MongoDB
dataset = [{"created_at": item["created_at"], "screen_name":item["screen_name"],
            "followers_count":item["followers_count"],
            "text": item["text"],
           } for item in col.find()]

***

## Transformação em DataFrame para manipulação com Pandas

In [ ]:
# Pacote Pandas
import pandas as pd
# Criação de um dataframe a partir do dataset 
dataframe = pd.DataFrame(dataset)
# Exibição do dataframe
dataframe

In [ ]:
# Exportação do DataFrame para um arquivo .csv
dataframe.to_csv('dados.csv', index=False)

***